In [ ]:


import os
import glob
import cv2
import numpy as np
import face_recognition
from matplotlib import pyplot as plt

def show_image(img, figsize=(10,6)):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=figsize)
    plt.axis('off')
    plt.imshow(img_rgb)
    plt.show()

In [ ]:

KNOWN_DIR = "known_faces"
known_encodings = []
known_names = []

for person_dir in os.listdir(KNOWN_DIR):
    person_path = os.path.join(KNOWN_DIR, person_dir)
    if not os.path.isdir(person_path):
        continue
    for img_path in glob.glob(os.path.join(person_path, "*")):
        img = face_recognition.load_image_file(img_path)
        encs = face_recognition.face_encodings(img)
        if len(encs) == 0:
            print(f"No face found in {img_path}, skipping.")
            continue
        known_encodings.append(encs[0])
        known_names.append(person_dir)

print(f"Loaded {len(known_encodings)} face encodings for {len(set(known_names))} people.")


In [ ]:

TEST_IMG = "test.jpg"   
image = cv2.imread(TEST_IMG)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


locations = face_recognition.face_locations(rgb, model="hog") 
encodings = face_recognition.face_encodings(rgb, locations)

for (top, right, bottom, left), face_encoding in zip(locations, encodings):
    matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.5)
    face_distances = face_recognition.face_distance(known_encodings, face_encoding)
    name = "Unknown"
    if True in matches:
        best_idx = np.argmin(face_distances)
        name = known_names[best_idx]
    cv2.rectangle(image, (left, top), (right, bottom), (0,255,0), 2)
    cv2.rectangle(image, (left, bottom-20), (right, bottom), (0,255,0), cv2.FILLED)
    cv2.putText(image, name, (left+2, bottom-4), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
show_image(image)


In [ ]:

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Could not open webcam")
while True:
    ret, frame = cap.read()
    if not ret:
        break
    small = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
    rgb_small = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
    locations = face_recognition.face_locations(rgb_small)
    encodings = face_recognition.face_encodings(rgb_small, locations)
    for (top, right, bottom, left), face_encoding in zip(locations, encodings):
        matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.5)
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        name = "Unknown"
        if True in matches:
            best_idx = np.argmin(face_distances)
            name = known_names[best_idx]
        top *= 2; right *= 2; bottom *= 2; left *= 2
        cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
        cv2.rectangle(frame, (left, bottom-20), (right, bottom), (0,255,0), cv2.FILLED)
        cv2.putText(frame, name, (left+2, bottom-4), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)

    cv2.imshow("Face Recognition (press q to quit)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
